In [1]:
from collections import OrderedDict
from os import getpid
from pprint import pprint
from shutil import rmtree
from time import sleep

from jcarbon.client import JCarbonClient
from jcarbon.tensorflow.callbacks import JCarbonEpochCallback

2024-05-22 19:03:15.085596: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 19:03:15.136750: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 19:03:15.136786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 19:03:15.138130: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 19:03:15.146102: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [4]:
jcarbon_callback = JCarbonEpochCallback()

In [5]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(units=32, activation='relu'))
model.add(keras.layers.Dense(10))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-2),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

2024-05-22 19:03:17.706163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 158 MB memory:  -> device: 0, name: Quadro P5000, pci bus id: 0000:18:00.0, compute capability: 6.1


In [6]:
result = model.fit(
    img_train,
    label_train,
    epochs=50,
    callbacks=[jcarbon_callback],
)

2024-05-22 19:03:28.029559: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 179.44MiB (rounded to 188160000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-05-22 19:03:28.029611: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-05-22 19:03:28.029632: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 12, Chunks in use: 12. 3.0KiB allocated for chunks. 3.0KiB in use in bin. 216B client-requested in use in bin.
2024-05-22 19:03:28.029647: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-05-22 19:03:28.029663: I external

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
signal = 'jcarbon.cpu.eflect.ProcessEnergy'
ax=None
for report in jcarbon_callback.reports:
    df = report.reset_index()
    df = df[df.signal == signal]
    df['elapsed'] = (df.start - df.start.min()).dt.total_seconds()
    df['rolled_value'] = df['value'].rolling(100).mean()
    ax = df.plot(x='elapsed', y='rolled_value', legend=False, figsize=(16, 9), ax=ax)